In [1]:
import kwargs as kwargs
# importing libraries
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import splitfolders

2022-10-13 20:25:40.047898: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-13 20:25:40.184069: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-13 20:25:40.188945: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-13 20:25:40.188958: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
# split into train, test and val
# splitfolders.ratio("data/tomato", output="data/tomato", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [3]:
# re-size all the images to this
IMAGE_SIZE = [256, 256, 3]

train_path = 'data/apple/train'
valid_path = 'data/apple/val'

In [4]:
# add preprocessing layer to the front of ResNet
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=IMAGE_SIZE
)

# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False


2022-10-13 20:25:43.051362: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-13 20:25:43.051382: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-13 20:25:43.051396: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (rohit-Mi-NoteBook-Ultra): /proc/driver/nvidia/version does not exist
2022-10-13 20:25:43.051525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# useful for getting number of classes
folders = glob('data/apple/train/*')
folders

['data/apple/train/Apple_Apple_scab',
 'data/apple/train/Apple_Black_rot',
 'data/apple/train/Apple_Cedar_apple_rust',
 'data/apple/train/Apple_healthy']

In [6]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [7]:
# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [8]:
# tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [9]:
train_datagen = ImageDataGenerator(rescale=1. / 256,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=0.2)

test_datagen = ImageDataGenerator(rescale=1. / 256)

training_set = train_datagen.flow_from_directory('data/apple/train',
                                                 target_size=(256, 256),
                                                 batch_size=32,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory('data/apple/val',
                                            target_size=(256, 256),
                                            batch_size=32,
                                            class_mode='categorical')

Found 4958 images belonging to 4 classes.
Found 618 images belonging to 4 classes.


In [10]:
# fit the model
apple_model = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/5
155/155 [==============================] - 276s 2s/step - loss: 2.7245 - accuracy: 0.4409 - val_loss: 1.4551 - val_accuracy: 0.3706
Epoch 2/5
155/155 [==============================] - 272s 2s/step - loss: 1.2063 - accuracy: 0.5353 - val_loss: 2.7678 - val_accuracy: 0.3544
Epoch 3/5
 10/155 [>.............................] - ETA: 3:46 - loss: 2.6258 - accuracy: 0.4187

KeyboardInterrupt: 